# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [3]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [4]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [5]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [8]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [9]:
directory_read_tool = DirectoryReadTool(directory='./instructions') # instructions to handle companies
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [12]:
directory_read_tool

DirectoryReadTool(name='List files in directory', description="List files in directory() - A tool that can be used to list ./instructions's content.", args_schema=<class 'crewai_tools.tools.directory_read_tool.directory_read_tool.FixedDirectoryReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7f08f3409f80>, directory='./instructions')

* enterprise_solutions_framework.md
* small_business_engagement.md
* tech_startups_outreach.md

Example:

# Personalized Outreach Instructions for Tech Startups

## Introduction
When engaging with tech startups, it's crucial to emphasize innovation, scalability, and how your solutions can aid in their growth and technological advancement. Startups are often looking for partners who understand their unique challenges and can offer flexible, cutting-edge solutions.

## Key Points to Address
- **Innovation:** Highlight how your products or services are at the forefront of technology.
- **Scalability:** Discuss how your solutions can grow with their business.
- **Support:** Emphasize the support and resources you provide, which are critical for startups.

## Template Message
Dear [Name],

Congratulations on [Recent Achievement/News]! At [Your Company], we're excited about the innovative work you're doing at [Startup Name]. Our [Product/Service] is designed with tech startups like yours in mind, offering [Key Feature] to help you [Benefit].

We understand the importance of scalability and flexibility in your journey. [Insert how your solution can be tailored to their current and future needs].

Let's explore how we can support [Startup Name]'s growth together.

Best,
[Your Name]


### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [13]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [14]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [15]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [16]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [17]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [18]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather information on DeepLearningAI, focusing on their company background, key personnel, recent milestones, and potential needs that align with our offerings. I must use all available data sources to compile a detailed profile for our engagement strategy.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company background"} 


Search results: Title: Deeplearning.ai -

Thought: I have gathered information on DeepLearningAI's company background, key personnel, recent milestones, and potential needs that align with our offerings. Now I need to compile a detailed report for our engagement strategy.

Final Answer: 
DeepLearningAI is an online learning platform founded in 2017 by Andrew Ng, a prominent figure in the field of machine learning and education. The company aims to provide world-class AI education accessible to a global audience. DeepLearningAI offers courses such as the Deep Learning Specialization and AI for Everyone, designed to help individuals start or advance their careers in AI.

Key personnel at DeepLearningAI include Andrew Ng, the founder, and CEO, known for his work in online education and democratizing deep learning. Other key members include Ted Greenwald, Bradley Reynolds, and Manfred Vogel, who hold important roles in strategy, AI, and deep learning.

Recent milestones for DeepLearningAI include advancements in AI models to becom

Thought: 
From the search results, I have gained insights into DeepLearningAI's company culture, which values contributions, opinions, and flexible remote work.

Final Answer:

Subject: Enhancing DeepLearningAI's Recent Milestone with Collaborative Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to reach out to congratulate you and the DeepLearningAI team on the recent launch of the "Generative AI for Everyone" course in collaboration with Coursera. This exciting initiative further solidifies DeepLearningAI's commitment to democratizing AI education globally.

At CrewAI, we share your passion for empowering individuals with the skills needed to navigate the ever-evolving landscape of artificial intelligence. Your dedication to providing world-class education aligns perfectly with our mission to support organizations in enhancing their learning platforms and delivering personalized experiences to students.

Based on our analysis of DeepLearningAI's recent achiev

- Display the final result as Markdown.

In [20]:
from IPython.display import Markdown
Markdown(result)

Subject: Enhancing DeepLearningAI's Recent Milestone with Collaborative Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to reach out to congratulate you and the DeepLearningAI team on the recent launch of the "Generative AI for Everyone" course in collaboration with Coursera. This exciting initiative further solidifies DeepLearningAI's commitment to democratizing AI education globally.

At CrewAI, we share your passion for empowering individuals with the skills needed to navigate the ever-evolving landscape of artificial intelligence. Your dedication to providing world-class education aligns perfectly with our mission to support organizations in enhancing their learning platforms and delivering personalized experiences to students.

Based on our analysis of DeepLearningAI's recent achievements and goals, we believe that our collaborative solutions can complement your efforts to innovate in the AI education space. By enhancing your online learning platform, improving course content delivery, and providing personalized learning experiences, we can help DeepLearningAI reach a broader audience and further advance AI education worldwide.

We are excited about the opportunity to work together and explore how our tools and services can support DeepLearningAI's mission. Our team is ready to offer collaboration opportunities, provide workshops or training sessions for your team, and engage in joint marketing efforts to promote our shared goal of advancing AI education.

I would love to schedule a call to discuss potential collaboration opportunities further. Please let me know a convenient time for you, and I will ensure to accommodate your schedule.

Thank you for your time and consideration. We look forward to the possibility of collaborating with DeepLearningAI to make a meaningful impact in the AI education space.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

Note: Please ensure to customize the email with your name and contact details before sending it to Andrew Ng.